In [ ]:
# Importing Libraries
import socket
import threading
import os
import time

In [ ]:
# Multi Threading Class
class NewClient(threading.Thread):
    def __init__(self,clientAddress,clientsocket):
        threading.Thread.__init__(self)
        self.address = clientAddress
        self.socket = clientsocket
        
        print ("New connection added: ", clientAddress)
        
    def run(self):
        print ("\nConnection from : ", self.address)
        msg = False
        
        rcvmesg = self.socket.recv(1024)
        rcvmsg = rcvmesg.decode("utf-8")
        
        print("Query From Master Server : " + rcvmsg)
        
        datatosend = ""
                
        path = './files/' # path of files location at sub server
        files = os.listdir(path)
        for fnamee in files:
            pathtofile = path + fnamee
            fname = fnamee.split(".")[0]
            if(fname == rcvmsg.split(".")[0]):
                msg = True
                
                # File Info
                info = []
                
                modefiedtime = os.path.getmtime(pathtofile)
                createdtime = os.path.getctime(pathtofile)
                size = os.stat(pathtofile).st_size

                
                print("File Name : " + str(fname))
                print("Last modified: %s" % time.ctime(modefiedtime))
                print("Created: %s" % time.ctime(createdtime))
                print("Size : " + str(size))

                info.append(str(fnamee))
                info.append(str(modefiedtime))
                info.append(str(createdtime))
                info.append(str(size))

                datatosend = "*-*" # to convert list to string
                datatosend = datatosend.join(info)
                datatosend = datatosend.encode()
                
        if(msg):
            print("File Founded")
            #Sending info to Server
            self.socket.send(datatosend)
        else:
            print("File not Founded")
            self.socket.send(bytes(str(msg),'UTF-8'))
        
#       Respondind back to client

#         s.close()

        

In [ ]:
#Listening for socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
s.bind((socket.gethostname(), 12345))

threadsList = []

print("Listening for Master Requests\n")
while True:
    s.listen(5)
    clientsocket, address = s.accept()
    
#     rcvmesg = clientsocket.recv(1024)
#     rcvmsg = rcvmesg.decode()
#     print("Receive : " + rcvmsg)
    
#     msg = ""
    
    newthread = NewClient(address,clientsocket) 
    newthread.start() 
    threadsList.append(newthread)
        
    
for thread in threadsList:
    thread.join()
